# FastCaddy Python 编程接口完整示例

本notebook展示如何使用FastCaddy Python版本进行域名管理，包括：

1. 域名配置状态检查
2. 安全的域名添加和删除
3. 批量域名操作
4. 通配符域名管理
5. 配置验证和故障排除

## 前置条件

- Caddy 服务器运行在 `localhost:2019`
- 已安装必要的Python依赖包

In [ ]:
# 导入必要的模块
import sys
import os
sys.path.append('..')

from fastcaddy.core import *
import time
from typing import Dict, List

## 1. 环境初始化和状态检查

In [ ]:
# 初始化FastCaddy环境
try:
    setup_caddy(local=True, install_trust=True)
    print("✅ FastCaddy环境初始化成功")
except Exception as e:
    print(f"⚠️ 初始化失败: {e}")
    print("这通常意味着Caddy服务器未运行")

In [ ]:
# 检查系统状态
def check_system_status():
    """检查FastCaddy系统状态"""
    status_checks = {
        "HTTP应用": "/apps/http",
        "TLS应用": "/apps/tls",
        "HTTP服务器": "/apps/http/servers",
        "TLS自动化": "/apps/tls/automation"
    }
    
    print("🔍 系统状态检查:")
    for name, path in status_checks.items():
        exists = has_path(path)
        icon = "✅" if exists else "❌"
        status = "已配置" if exists else "未配置"
        print(f"  {icon} {name}: {status}")
    
    return {name: has_path(path) for name, path in status_checks.items()}

system_status = check_system_status()

## 2. 域名状态检查功能

In [ ]:
def check_domain_status(domain: str) -> bool:
    """检查单个域名配置状态"""
    exists = has_id(domain)
    icon = "✅" if exists else "❌"
    status = "已配置" if exists else "未配置"
    print(f"{icon} 域名 {domain}: {status}")
    return exists

def batch_check_domains(domains: List[str]) -> Dict[str, bool]:
    """批量检查域名状态"""
    print(f"\n🔍 批量检查 {len(domains)} 个域名:")
    results = {}
    for domain in domains:
        results[domain] = check_domain_status(domain)
    return results

# 测试域名列表
test_domains = [
    "api.example.com",
    "web.example.com", 
    "admin.example.com",
    "nonexistent.example.com"
]

initial_status = batch_check_domains(test_domains)

## 3. 安全的域名添加功能

In [ ]:
def safe_add_domain(domain: str, target: str, force: bool = False) -> bool:
    """安全添加域名配置"""
    print(f"\n🔍 准备添加域名: {domain} -> {target}")
    
    # 检查域名是否已存在
    if has_id(domain):
        if force:
            print(f"⚠️ 域名 {domain} 已存在，强制更新模式")
            try:
                del_id(domain)
                print(f"🗑️ 删除现有配置成功")
                time.sleep(0.1)  # 短暂延迟
            except Exception as e:
                print(f"❌ 删除现有配置失败: {e}")
                return False
        else:
            print(f"❌ 域名 {domain} 已存在，请使用 force=True 强制更新")
            return False
    
    # 添加新配置
    try:
        add_reverse_proxy(domain, target)
        print(f"✅ 成功添加域名 {domain}")
        return True
    except Exception as e:
        print(f"❌ 添加域名失败: {e}")
        return False

# 演示添加域名
domain_configs = {
    "api.example.com": "localhost:8080",
    "web.example.com": "localhost:3000",
    "admin.example.com": "localhost:9000"
}

print("📝 开始添加域名配置...")
add_results = {}
for domain, target in domain_configs.items():
    add_results[domain] = safe_add_domain(domain, target)

# 显示添加结果
success_count = sum(1 for success in add_results.values() if success)
print(f"\n📊 添加结果: {success_count}/{len(domain_configs)} 个域名添加成功")

## 4. 验证添加结果

In [ ]:
# 验证所有域名是否成功添加
print("✅ 验证添加结果:")
verification_results = batch_check_domains(list(domain_configs.keys()))

# 统计结果
configured_count = sum(1 for exists in verification_results.values() if exists)
print(f"\n📈 配置状态: {configured_count}/{len(domain_configs)} 个域名已正确配置")

## 5. 域名配置更新

In [ ]:
def update_domain_config(domain: str, new_target: str) -> bool:
    """更新域名配置"""
    print(f"\n🔄 更新域名配置: {domain} -> {new_target}")
    return safe_add_domain(domain, new_target, force=True)

# 演示更新配置
print("🔄 演示配置更新:")
update_success = update_domain_config("api.example.com", "localhost:8081")

if update_success:
    print("✅ 配置更新成功")
else:
    print("❌ 配置更新失败")

## 6. 通配符域名管理

In [ ]:
def manage_wildcard_domain(base_domain: str, subdomains: Dict[str, str]):
    """管理通配符域名和子域名"""
    wildcard_id = f"wildcard-{base_domain}"
    
    print(f"🌟 管理通配符域名: *.{base_domain}")
    
    # 添加通配符路由
    if not has_id(wildcard_id):
        try:
            add_wildcard_route(base_domain)
            print(f"✅ 添加通配符域名 *.{base_domain}")
        except Exception as e:
            print(f"❌ 添加通配符域名失败: {e}")
            return False
    else:
        print(f"ℹ️ 通配符域名 *.{base_domain} 已存在")
    
    # 添加子域名
    print(f"\n📋 添加子域名到 {base_domain}:")
    subdomain_results = {}
    
    for subdomain, port in subdomains.items():
        full_domain = f"{subdomain}.{base_domain}"
        
        if not has_id(full_domain):
            try:
                add_sub_reverse_proxy(base_domain, subdomain, port)
                print(f"  ✅ {full_domain} -> localhost:{port}")
                subdomain_results[full_domain] = True
            except Exception as e:
                print(f"  ❌ {full_domain} 添加失败: {e}")
                subdomain_results[full_domain] = False
        else:
            print(f"  ⚠️ {full_domain} 已存在")
            subdomain_results[full_domain] = True
    
    return subdomain_results

# 演示通配符域名管理
wildcard_base = "dev.local"
wildcard_subdomains = {
    "app": "8090",
    "api": "8091",
    "admin": "8092",
    "monitoring": "8093"
}

wildcard_results = manage_wildcard_domain(wildcard_base, wildcard_subdomains)

## 7. 验证通配符域名配置

In [ ]:
# 检查所有子域名状态
subdomain_list = [f"{sub}.{wildcard_base}" for sub in wildcard_subdomains.keys()]
wildcard_status = batch_check_domains(subdomain_list)

# 检查通配符路由本身
wildcard_id = f"wildcard-{wildcard_base}"
wildcard_route_exists = check_domain_status(wildcard_id)

print(f"\n📊 通配符域名配置总结:")
configured_subs = sum(1 for exists in wildcard_status.values() if exists)
print(f"  子域名: {configured_subs}/{len(subdomain_list)} 个已配置")
print(f"  通配符路由: {'已配置' if wildcard_route_exists else '未配置'}")

## 8. 安全的域名删除功能

In [ ]:
def safe_delete_domain(domain: str, verify: bool = True) -> bool:
    """安全删除域名配置"""
    print(f"\n🗑️ 准备删除域名: {domain}")
    
    # 检查域名是否存在
    if not has_id(domain):
        print(f"⚠️ 域名 {domain} 不存在，无需删除")
        return True
    
    # 删除域名
    try:
        del_id(domain)
        print(f"✅ 删除命令执行成功")
        
        # 验证删除结果
        if verify:
            time.sleep(0.1)  # 短暂延迟
            if has_id(domain):
                print(f"❌ 验证失败: 域名 {domain} 仍然存在")
                return False
            else:
                print(f"✅ 验证成功: 域名 {domain} 已完全删除")
        
        return True
    except Exception as e:
        print(f"❌ 删除域名失败: {e}")
        return False

def batch_delete_domains(domains: List[str]) -> Dict[str, bool]:
    """批量删除域名"""
    print(f"\n🗑️ 批量删除 {len(domains)} 个域名:")
    results = {}
    for domain in domains:
        results[domain] = safe_delete_domain(domain)
    return results

# 演示删除部分域名
domains_to_delete = ["admin.example.com", "monitoring.dev.local"]
delete_results = batch_delete_domains(domains_to_delete)

success_deletes = sum(1 for success in delete_results.values() if success)
print(f"\n📊 删除结果: {success_deletes}/{len(domains_to_delete)} 个域名删除成功")

## 9. 最终状态检查和总结

In [ ]:
# 检查所有配置的最终状态
print("📈 最终配置状态检查:")

all_domains = list(domain_configs.keys()) + subdomain_list + [wildcard_id]
final_status = batch_check_domains(all_domains)

# 分类统计
active_domains = [domain for domain, exists in final_status.items() if exists]
inactive_domains = [domain for domain, exists in final_status.items() if not exists]

print(f"\n📊 配置总结:")
print(f"  🟢 活跃域名: {len(active_domains)} 个")
for domain in active_domains:
    print(f"    - {domain}")

print(f"  🔴 未配置域名: {len(inactive_domains)} 个")
for domain in inactive_domains:
    print(f"    - {domain}")

## 10. 配置信息查看

In [ ]:
# 获取详细配置信息
def get_detailed_config_info():
    """获取详细的配置信息"""
    print("🔍 获取详细配置信息:")
    
    try:
        # 获取HTTP服务器配置
        if has_path("/apps/http/servers"):
            http_servers = gcfg("/apps/http/servers")
            print(f"  📡 HTTP服务器: {len(http_servers)} 个")
            for server_name in http_servers.keys():
                print(f"    - {server_name}")
        else:
            print("  📡 HTTP服务器: 未配置")
        
        # 获取根配置
        root_config = gcfg("/")
        if 'apps' in root_config:
            apps = root_config['apps']
            print(f"  🚀 应用程序: {len(apps)} 个")
            for app_name in apps.keys():
                print(f"    - {app_name}")
        
    except Exception as e:
        print(f"  ❌ 获取配置信息失败: {e}")

get_detailed_config_info()

## 11. 清理测试配置（可选）

In [ ]:
# 清理所有测试配置
def cleanup_all_test_configs():
    """清理所有测试配置"""
    print("🧹 开始清理所有测试配置...")
    
    # 清理普通域名
    normal_domains = ["api.example.com", "web.example.com"]
    batch_delete_domains(normal_domains)
    
    # 清理子域名
    remaining_subdomains = [f"{sub}.{wildcard_base}" for sub in wildcard_subdomains.keys() 
                           if f"{sub}.{wildcard_base}" not in domains_to_delete]
    if remaining_subdomains:
        batch_delete_domains(remaining_subdomains)
    
    # 清理通配符路由
    safe_delete_domain(wildcard_id)
    
    print("✅ 清理完成")

# 运行清理（注释掉以保留配置用于测试）
# cleanup_all_test_configs()

## 总结

本notebook演示了FastCaddy Python编程接口的主要功能：

### 核心功能
1. **状态检查**: `has_id()`, `has_path()`, `gcfg()`
2. **域名添加**: `add_reverse_proxy()`, `add_wildcard_route()`, `add_sub_reverse_proxy()`
3. **域名删除**: `del_id()`
4. **配置管理**: `setup_caddy()`, `gcfg()`

### 最佳实践
1. **操作前检查**: 总是使用 `has_id()` 检查域名状态
2. **安全更新**: 更新配置时先删除再添加
3. **验证结果**: 操作后验证配置是否正确应用
4. **错误处理**: 使用 try-except 处理可能的异常
5. **批量操作**: 对多个域名使用批量处理函数

### 使用技巧
- 使用通配符域名简化子域名管理
- 定期检查系统状态确保配置一致性
- 开发和生产环境使用不同域名前缀
- 及时清理测试配置避免配置污染